## Energy saved from recycling
<p>Did you know that recycling saves energy by reducing or eliminating the need to make materials from scratch? For example, aluminum can manufacturers can skip the energy-costly process of producing aluminum from ore by cleaning and melting recycled cans. Aluminum is classified as a non-ferrous metal.</p>
<p>Singapore has an ambitious goal of becoming a zero-waste nation. The amount of waste disposed of in Singapore has increased seven-fold over the last 40 years. At this rate, Semakau Landfill, Singapore’s only landfill, will run out of space by 2035. Making matters worse, Singapore has limited land for building new incineration plants or landfills.</p>
<p>The government would like to motivate citizens by sharing the total energy that the combined recycling efforts have saved every year. They have asked you to help them.</p>
<p>You have been provided with three datasets. The data come from different teams, so the names of waste types may differ.</p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/wastestats.csv - Recycling statistics per waste type for the period 2003 to 2017</b>
    </div>
    <div>Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>waste_type: </b>The type of waste recycled.</li>
    <li><b>waste_disposed_of_tonne: </b>The amount of waste that could not be recycled (in metric tonnes).</li>
    <li><b>total_waste_recycle_tonne: </b>The amount of waste that could be recycled (in metric tonnes).</li>
    <li><b>total_waste_generated: </b>The total amount of waste collected before recycling (in metric tonnes).</li>
    <li><b>recycling_rate: </b>The amount of waste recycled per tonne of waste generated.</li>
    <li><b>year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/2018_2019_waste.csv - Recycling statistics per waste type for the period 2018 to 2019</b>
    </div>
    <div> Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>Waste Type: </b>The type of waste recycled.</li>
    <li><b>Total Generated: </b>The total amount of waste collected before recycling (in thousands of metric tonnes).</li> 
    <li><b>Total Recycled: </b>The amount of waste that could be recycled. (in thousands of metric tonnes).</li>
    <li><b>Year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/energy_saved.csv -  Estimations of the amount of energy saved per waste type in kWh</b>
    </div>
<ul>
    <li><b>material: </b>The type of waste recycled.</li>
    <li><b>energy_saved: </b>An estimate of the energy saved (in kiloWatt hour) by recycling a metric tonne of waste.</li> 
    <li><b>crude_oil_saved: </b>An estimate of the number of barrels of oil saved by recycling a metric tonne of waste.</li>
</ul>

</div>
<pre><code>
</code></pre>

In [28]:
import pandas as pd
import numpy as np

# Recycling statistics per waste type for the period 2003 to 2017
df1 = pd.read_csv('datasets/wastestats.csv')

# Recycling statistics per waste type for the period 2018 to 2019
df2 = pd.read_csv('datasets/2018_2019_waste.csv')

# Estimations of the amount of energy saved per waste type in kWh
df3 = pd.read_csv('datasets/energy_saved.csv')

# Dropping cols that will not be used from df1
df1.drop(['waste_disposed_of_tonne', 'recycling_rate'], inplace=True, axis=1)

# Changing measurements of df2 to fit df1
df2["Total Generated ('000 tonnes)"] = df2["Total Generated ('000 tonnes)"].astype(float)*1000.0
df2["Total Recycled ('000 tonnes)"] = df2["Total Recycled ('000 tonnes)"].astype(float)*1000.0

# Changing type Total Generated Waste col in df1 to float
df1["total_waste_generated_tonne"] = df1["total_waste_generated_tonne"].astype(float)

# Changing col order of df1 to fit df2
cols = df1.columns.tolist()
cols = [cols[0], cols[2], cols[1], cols[3]]
df1 = df1[cols]

# Change to apropriate and same headers for both df1 and df2
df1.rename({'waste_type': 'Waste Type', 'total_waste_generated_tonne': 'Generated (tonne)', 'total_waste_recycled_tonne': 'Recycled (tonne)', 'year': 'Year'}, inplace=True, axis=1)
df2.rename({"Total Generated ('000 tonnes)": 'Generated (tonne)', "Total Recycled ('000 tonnes)": 'Recycled (tonne)'}, inplace=True, axis=1)


# Stack df1 and df2 on top of each other 
df = pd.concat([df1, df2])

# Limit df to include only data between 2015 and 2019
years = ['2015', '2016', '2017', '2018', '2019']
df = df.loc[df['Year'].isin(years)]

# Change any spelling of metals to 'Ferrous metals' and 'Non-ferrous metals'
# Currently: Non-Ferrous Metal, Non-ferrous Metal, Non-ferrous metal, Ferrous metals, Ferrous Metal ...
df.loc[df['Waste Type'].str.lower().str.contains("non-ferrous metal"), 'Waste Type'] = 'Non-ferrous metals'
df.loc[df['Waste Type'].str.lower().isin(["ferrous metal", "ferrous metals"]), 'Waste Type'] = 'Ferrous metals'

# Change any spelling of plastic to 'Plastics'
df.loc[df['Waste Type'].str.lower().str.contains("plastic"),'Waste Type'] = 'Plastics'

# Limit df to include only: Plastic, Glass, Ferrous Metal, and Non-Ferrous Metal
waste_types = ['Plastics', 'Glass', 'Ferrous metals', 'Non-ferrous metals']
df = df.loc[df['Waste Type'].isin(waste_types)]

# Sort by the year col
df = df.sort_values('Year', ascending=True)

# Add energy saved per tonne col
df.loc[df['Waste Type'] == 'Plastics', 'Energy (p/t)'] = float(df3.loc[3][1][0:-4])
df.loc[df['Waste Type'] == 'Glass', 'Energy (p/t)'] = float(df3.loc[3][2][0:-4])
df.loc[df['Waste Type'] == 'Ferrous metals', 'Energy (p/t)'] = float(df3.loc[3][3][0:-4])
df.loc[df['Waste Type'] == 'Non-ferrous metals', 'Energy (p/t)'] = float(df3.loc[3][4][0:-4])

# Calculate and add total energy saved col
df['Total Energy Saved (Kwh)'] = df['Recycled (tonne)'] * df['Energy (p/t)']

# Drop a few cols we not needed anymore
df.drop('Energy (p/t)', axis=1, inplace=True)
df.drop('Recycled (tonne)', axis=1, inplace=True)
df.drop('Generated (tonne)', axis=1, inplace=True)
df.drop('Waste Type', axis=1, inplace=True)

# Sum up total saved energy per year and 
df = df.groupby('Year')['Total Energy Saved (Kwh)'].sum().reset_index().rename(columns={'Total Energy Saved (Kwh)':'total_energy_saved','Year' : 'year'})

# Set year col as index
df = df.set_index('year')

# Assign the proper col names requested.
#df.rename({'Year': 'year', 'Total Energy Saved (Kwh)': 'total_energy_saved'}, inplace=True, axis=1)
#df.columns = ['total_energy_saved']


print(df)



                
# Create the final dataframe
#annual_energy_savings = pd.DataFrame()

# Assigning df to the new dataframe with the name requested
#annual_energy_savings = df

# Assign the proper col names requested.
#annual_energy_savings.columns = ['year', 'total_energy_saved']

# Set year col as index
#annual_energy_savings = annual_energy_savings.set_index('year')

#print(annual_energy_savings)


      total_energy_saved
year                    
2015        3.435929e+09
2016        2.554433e+09
2017        2.470596e+09
2018        2.698130e+09
2019        2.765440e+09
